<center><img src=https://raw.githubusercontent.com/feast-dev/feast/master/docs/assets/feast_logo.png width=400/></center>

# Deploying the Feature Store

### Introduction

Feast enables AI/ML teams to serve (and consume) features via feature stores. In this notebook, we will configure the feature stores and feature definitions, and deploy a Feast feature store server. We will also materialize (move) data from the offline store to the online store.

In Feast, offline stores support pulling large amounts of data for model training using tools like Redshift, Snowflake, Bigquery, and Spark. In contrast, the focus of Feast online stores is feature serving in support of model inference, using tools like Redis, Snowflake, PostgreSQL, and SQLite.

In this notebook, we will setup a file-based (Dask) offline store and SQLite online store. The online store will be made available through the Feast server.

This notebook assumes that you have prepared the data by running the notebook [01_Credit_Risk_Data_Prep.ipynb](01_Credit_Risk_Data_Prep.ipynb).

### Setup

*The following code assumes that you have read the example README.md file, and that you have setup an environment where the code can be run. Please make sure you have addressed the prerequisite needs.*

In [3]:
# Imports
import re
import sys
import time
import signal
import sqlite3
import subprocess
import datetime as dt
from feast import FeatureStore

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [8]:
!pip install feast

### Feast Feature Store Configuration

For model training, we usually don't need (or want) a constantly running feature server. All we need is the ability to efficiently query and pull all of the training data at training time. In contrast, during model serving we need servers that are always ready to supply feature records in response to application requests.

This training-serving dichotomy is reflected in Feast using "offline" and "online" stores. Offline stores are configured to work with database technologies typically used for training, while online stores are configured to use storage and streaming technologies that are popular for feature serving.

We need to create a `feature_store.yaml` config file to tell feast the structure we want in our offline and online feature stores. Below, we write the configuration for a local "Dask" offline store and local SQLite online store. We give the feature store a project name of `loan_applications`, and provider `local`. The registry is where the feature store will keep track of feature definitions and online store updates; we choose a file location in this case.

See the [feature_store.yaml](https://docs.feast.dev/reference/feature-repository/feature-store-yaml) documentation for further details.

In [5]:
%%writefile Feature_Store/feature_store.yaml

project: loan_applications
registry: data/registry.db
provider: local
offline_store:
    type: dask
online_store:
    type: sqlite
    path: data/online_store.db
entity_key_serialization_version: 3

Writing Feature_Store/feature_store.yaml


### Feature Definitions

We also need to create feature definitions and other feature constructs in a python file, which we name `feature_definitions.py`. For our purposes, we define the following:

- Data Source: connections to data storage or data-producing endpoints
- Entity: primary key fields which can be used for joining data
- FeatureView: collections of features from a data source

For more information on these, see the [Concepts](https://docs.feast.dev/getting-started/concepts) section of the Feast documentation.

In [17]:
%%writefile Feature_Store/feature_definitions.py

# Imports
import os
from pathlib import Path
from feast import (
    FileSource,
    Entity,
    FeatureView,
    Field,
    FeatureService,
    ValueType # Import ValueType
)
from feast.types import Float32, String # Import String type from feast.types
from feast.data_format import ParquetFormat

CURRENT_DIR = os.path.abspath(os.curdir)

# Data Sources
# A data source tells Feast where the data lives
data_a = FileSource(
    file_format=ParquetFormat(),
    path="data/data_a.parquet"
)
data_b = FileSource(
    file_format=ParquetFormat(),
    path="data/data_b.parquet"
)

# Entity
# An entity tells Feast the column it can use to join tables
loan_id = Entity(
    name = "loan_id",
    join_keys = ["ID"],
    value_type=ValueType.STRING # Keep ValueType.STRING for Entity
)

# Feature views
# A feature view is how Feast groups features
features_a = FeatureView(
    name="data_a",
    entities=[loan_id],
    schema=[
        Field(name="checking_status", dtype=String),
        Field(name="duration", dtype=Float32),
        Field(name="credit_history", dtype=String),
        Field(name="purpose", dtype=String),
        Field(name="credit_amount", dtype=Float32),
        Field(name="savings_status", dtype=String),
        Field(name="employment", dtype=String),
        Field(name="installment_commitment", dtype=Float32),
        Field(name="personal_status", dtype=String),
        Field(name="other_parties", dtype=String),
    ],
    source=data_a
)
features_b = FeatureView(
    name="data_b",
    entities=[loan_id],
    schema=[
        Field(name="residence_since", dtype=Float32),
        Field(name="property_magnitude", dtype=String),
        Field(name="age", dtype=Float32),
        Field(name="other_payment_plans", dtype=String),
        Field(name="housing", dtype=String),
        Field(name="existing_credits", dtype=Float32),
        Field(name="job", dtype=String),
        Field(name="num_dependents", dtype=Float32),
        Field(name="own_telephone", dtype=String),
        Field(name="foreign_worker", dtype=String),
    ],
    source=data_b
)

# Feature Service
# a feature service in Feast represents a logical group of features
loan_fs = FeatureService(
    name="loan_fs",
    features=[features_a, features_b]
)

Overwriting Feature_Store/feature_definitions.py


In [12]:
# Create dummy data for data_a.parquet and data_b.parquet
import pandas as pd
import numpy as np
import os
import datetime as dt

# Ensure the data directory exists
os.makedirs("Feature_Store/data", exist_ok=True)

# Create dummy data for features_a
data = {
    "ID": [str(x) for x in range(1, 1001)], # Convert ID to string for consistency with type=String
    "checking_status": np.random.choice(["0<=X<200", "<0", "no checking"], 1000),
    "duration": np.random.randint(4, 72, 1000).astype(float),
    "credit_history": np.random.choice(["existing paid", "critical/other existing credit", "all paid"], 1000),
    "purpose": np.random.choice(["car", "radio/tv", "furniture/equipment", "education"], 1000),
    "credit_amount": np.random.randint(250, 15000, 1000).astype(float),
    "savings_status": np.random.choice(["<100", "100<=X<500", "no known savings"], 1000),
    "employment": np.random.choice(["<1 year", "1<=X<4 years", ">=7 years"], 1000),
    "installment_commitment": np.random.randint(1, 5, 1000).astype(float),
    "personal_status": np.random.choice(["male single", "female div/dep/mar", "male mar/wid"], 1000),
    "other_parties": np.random.choice(["none", "guarantor", "co applicant"], 1000),
    "event_timestamp": pd.to_datetime([dt.datetime(2023, 9, 24, 12, 0, 0) + dt.timedelta(hours=i) for i in range(1000)])
}
df_a = pd.DataFrame(data)
df_a.to_parquet("Feature_Store/data/data_a.parquet", index=False)

# Create dummy data for features_b
data = {
    "ID": [str(x) for x in range(1, 1001)],
    "residence_since": np.random.randint(1, 5, 1000).astype(float),
    "property_magnitude": np.random.choice(["car", "real estate", "life insurance", "no known property"], 1000),
    "age": np.random.randint(18, 75, 1000).astype(float),
    "other_payment_plans": np.random.choice(["none", "bank", "stores"], 1000),
    "housing": np.random.choice(["own", "rent", "for free"], 1000),
    "existing_credits": np.random.randint(1, 5, 1000).astype(float),
    "job": np.random.choice(["skilled", "unskilled resident", "high qualif/self emp/mgmt"], 1000),
    "num_dependents": np.random.randint(1, 3, 1000).astype(float),
    "own_telephone": np.random.choice(["yes", "no"], 1000),
    "foreign_worker": np.random.choice(["yes", "no"], 1000),
    "event_timestamp": pd.to_datetime([dt.datetime(2023, 9, 24, 12, 0, 0) + dt.timedelta(hours=i) for i in range(1000)])
}
df_b = pd.DataFrame(data)
df_b.to_parquet("Feature_Store/data/data_b.parquet", index=False)

print("Dummy data files data_a.parquet and data_b.parquet created in Feature_Store/data/")

Dummy data files data_a.parquet and data_b.parquet created in Feature_Store/data/


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


### Applying the Configuration and Definitions

Now that we have our feature store configuration (`feature_store.yaml`) and feature definitions (`feature_definitions.py`), we are ready to "apply" them. The `feast apply` command creates a registry file (`Feature_Store/data/registry.db`) and sets up data connections; in this case, it creates a SQLite database (`Feature_Store/data/online_store.db`).

In [18]:
# Run 'feast apply' in the Feature_Store directory
!feast --chdir ./Feature_Store apply

No project found in the repository. Using project name loan_applications defined in feature_store.yaml
Applying changes for project loan_applications
Created project loan_applications
Created entity loan_id
Created feature view data_a
Created feature view data_b
Created feature service loan_fs

Created sqlite table loan_applications_data_a
Created sqlite table loan_applications_data_b



In [19]:
# List the Feature_Store/data/ directory to see newly created files
!ls -nlh Feature_Store/data/

total 92K
-rw-r--r-- 1 0 0  31K Nov 25 18:23 data_a.parquet
-rw-r--r-- 1 0 0  25K Nov 25 18:23 data_b.parquet
-rw-r--r-- 1 0 0  28K Nov 25 18:25 online_store.db
-rw-r--r-- 1 0 0 2.3K Nov 25 18:25 registry.db


Note that while `feast apply` set up the `sqlite` online database, `online_store.db`, no data has been added to the online database as of yet. We can verify this by connecting with the `sqlite3` library.

In [20]:
# Connect to sqlite database
conn = sqlite3.connect("Feature_Store/data/online_store.db")
cursor = conn.cursor()
# Query table data (3 tables)
print(
    "Online Store Tables:           ",
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
)
print(
    "loan_applications_data_a data: ",
    cursor.execute("SELECT * FROM loan_applications_data_a").fetchall()
)
print(
    "loan_applications_data_b data: ",
    cursor.execute("SELECT * FROM loan_applications_data_b").fetchall()
)
conn.close()

Online Store Tables:            [('loan_applications_data_a',), ('loan_applications_data_b',)]
loan_applications_data_a data:  []
loan_applications_data_b data:  []


Since we have used `feast apply` to create the registry, we can now use the Feast Python SDK to interact with our new feature store. To see other possible commands see the [Feast Python SDK documentation](https://rtd.feast.dev/en/master/).

In [21]:
# Get feature store config
store = FeatureStore(repo_path="./Feature_Store")
store.config

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


RepoConfig(project='loan_applications', project_description=None, provider='local', registry_config='data/registry.db', online_config={'type': 'sqlite', 'path': 'data/online_store.db'}, auth={'type': 'no_auth'}, offline_config={'type': 'dask'}, batch_engine_config='local', feature_server=None, flags=None, repo_path=PosixPath('Feature_Store'), entity_key_serialization_version=3, coerce_tz_aware=True, materialization_config=MaterializationConfig(pull_latest_features=False))

In [22]:
# List feature views
feature_views = store.list_batch_feature_views()
for fv in feature_views:
    print(f"Feature view: {fv.name}  |  Features: {fv.features}")

Feature view: data_a  |  Features: [Field(
    name='checking_status',
    dtype=<PrimitiveFeastType.STRING: 2>,
    description='',
    tags={}
    vector_index=False
    vector_length=0
    vector_search_metric=''
), Field(
    name='duration',
    dtype=<PrimitiveFeastType.FLOAT32: 6>,
    description='',
    tags={}
    vector_index=False
    vector_length=0
    vector_search_metric=''
), Field(
    name='credit_history',
    dtype=<PrimitiveFeastType.STRING: 2>,
    description='',
    tags={}
    vector_index=False
    vector_length=0
    vector_search_metric=''
), Field(
    name='purpose',
    dtype=<PrimitiveFeastType.STRING: 2>,
    description='',
    tags={}
    vector_index=False
    vector_length=0
    vector_search_metric=''
), Field(
    name='credit_amount',
    dtype=<PrimitiveFeastType.FLOAT32: 6>,
    description='',
    tags={}
    vector_index=False
    vector_length=0
    vector_search_metric=''
), Field(
    name='savings_status',
    dtype=<PrimitiveFeastType.S

### Deploying the Feature Store Servers

If you wish to share a feature store with your team, Feast provides feature servers. To spin up an offline feature server process, we can use the `feast serve_offline` command, while to spin up a Feast online feature server, we use the `feast serve` command.

Let's spin up an offline and an online server that we can use in the subsequent notebooks to get features during model training and model serving. We will run both servers as background processes, that we can communicate with in the other notebooks.

First, we write a helper function to extract the first few printed log lines (so we can print it in the notebook cell output).

In [23]:
# TimeoutError class
class TimeoutError(Exception):
    pass

# TimeoutError raise function
def timeout():
    raise TimeoutError("timeout")

# Get first few log lines function
def print_first_proc_lines(proc, wait):
    '''Given a process, `proc`, read and print output lines until they stop
    comming (waiting up to `wait` seconds for new lines to appear)'''
    lines = ""
    while True:
        signal.signal(signal.SIGALRM, timeout)
        signal.alarm(wait)
        try:
            lines += proc.stderr.readline()
        except:
            break
    if lines:
        print(lines, file=sys.stderr)

Launch the offline server with the command `feast --chdir ./Feature_Store serve_offline`.

In [24]:
# Feast offline server process
offline_server_proc = subprocess.Popen(
    "feast --chdir ./Feature_Store serve_offline 2>&2 & echo $! > server_proc.txt",
    shell=True,
    text=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    bufsize=0
)
print_first_proc_lines(offline_server_proc, 2)

The tail end of the command above, `2>&2 & echo $! > server_proc.txt`, captures log messages (in the offline case there are none), and writes the process PID to the file `server_proc.txt` (we will use this in the cleanup notebook, [05_Credit_Risk_Cleanup.ipynb](05_Credit_Risk_Cleanup.ipynb)).

Next, launch the online server with the command `feast --chdir ./Feature_Store serve`.

In [25]:
# Feast online server (master and worker) processes
online_server_proc = subprocess.Popen(
    "feast --chdir ./Feature_Store serve 2>&2 & echo $! >> server_proc.txt",
    shell=True,
    text=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    bufsize=0
)
print_first_proc_lines(online_server_proc, 3)

Note that the output helpfully let's us know that the online server is "Listening at: http://127.0.0.1:6566" (the default host:port).

List the running processes to verify they are up.

In [26]:
# List running Feast processes (paths redacted)
running_procs = !ps -ef | grep feast | grep serve

for line in running_procs:
    redacted = re.sub(r'/*[^\s]*(?P<cmd>(python )|(feast ))', r'**/\g<cmd>', line)
    print(redacted)

root        4821       1 71 18:26 ?        00:00:08 /usr/bin/python3 **/feast --chdir ./Feature_Store serve_offline
root        4847       1 83 18:26 ?        00:00:04 /usr/bin/python3 **/feast --chdir ./Feature_Store serve
root        4876    1033  0 18:26 ?        00:00:00 /bin/bash -c ps -ef | grep **/feast | grep serve


Note that there are two process for the online server (master and worker).

### Materialize Features to the Online Store

At this point, there is no data in the online store yet. Let's use the SDK feature store object (that we created above) to "materialize" data; this is Feast lingo for moving/updating data from the offline store to the online store.

In [27]:
# Materialize
# Recall that we mocked the outcome data to have timestamps from
# 'Tue Sep 24 12:00:00 2023'out to "Wed Oct  9 12:00:00 2023"
# The loan outcome timestamps were then lagged by 30-90 days (which is Jan 7 12:00:00 2024)
res = store.materialize(
    start_date=dt.datetime(2023,9,24,12,0,0),
    end_date=dt.datetime(2024,1,7,12,0,0)
)

Materializing 2 feature views from 2023-09-24 12:00:00+00:00 to 2024-01-07 12:00:00+00:00 into the sqlite online store.

data_a:


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


data_b:


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Now, we can query the SQLite database again and see data in the response!

In [28]:
# Query the online store database to verify materialized data
conn = sqlite3.connect("Feature_Store/data/online_store.db")
cursor = conn.cursor()
print(
    "loan_applications_data_a data: ",
    cursor.execute("SELECT * FROM loan_applications_data_a LIMIT 2").fetchall()
)
print(
    "loan_applications_data_b data: ",
    cursor.execute("SELECT * FROM loan_applications_data_b LIMIT 2").fetchall()
)
conn.close()

loan_applications_data_a data:  [(b'\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00ID\x02\x00\x00\x00\x04\x00\x00\x001000', 'checking_status', b'\x12\x0bno checking', None, 1699153200, None), (b'\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00ID\x02\x00\x00\x00\x04\x00\x00\x001000', 'duration', b'5\x00\x00\x88A', None, 1699153200, None)]
loan_applications_data_b data:  [(b'\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00ID\x02\x00\x00\x00\x04\x00\x00\x001000', 'residence_since', b'5\x00\x00\x00@', None, 1699153200, None), (b'\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00ID\x02\x00\x00\x00\x04\x00\x00\x001000', 'property_magnitude', b'\x12\x0elife insurance', None, 1699153200, None)]


Note that the data is stored in binary strings, which is part of Feast's optimization for online queries. To get human-readable data, use the `get-online-features` REST API command, which returns a JSON response.

In [29]:
# curl command to online server to get data from the online store
cmd = """http://localhost:6566/get-online-features \
    -d '{
            "feature_service": "loan_fs",
            "entities": {"ID": [18, 764]}
        }'
"""

response = !curl -X POST {cmd}

In [30]:
response

['curl: (7) Failed to connect to localhost port 6566 after 0 ms: Connection refused']

The `curl` command gave us a quick validation. In the [04_Credit_Risk_Model_Serving.ipynb](04_Credit_Risk_Model_Serving.ipynb) notebook, we'll use the Python `requests` library to handle the query better.

Now that the feature stores and their respective servers have been configured and deployed, we can proceed to train an AI model in [03_Credit_Risk_Model_Training.ipynb](03_Credit_Risk_Model_Training.ipynb).